In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sys
sys.path.append('/Users/david/miniconda3/lib/python3.8/site-packages')

import cv2 as cv

from ipywidgets import interact
import ipywidgets as widgets

%matplotlib auto

Using matplotlib backend: MacOSX


In [2]:
def filtre(w, h, dim):
    
    xpx, ypx = dim[0], dim[1]
    almostSquare = w > 0 and .9 < h/w < 1.1
    relevantDim = (.05*xpx < w < .7*xpx) and (.05*ypx < h < .7*ypx)
    
    return almostSquare and relevantDim

In [3]:
def drawRectangle(image, show=True, unrot=True):
    dim = image.shape
    cimg = cv.cvtColor(image, cv.COLOR_GRAY2RGB)
    contours, h = cv.findContours(image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    rect = []
    
    total = 0
    good = 0
    
    for cnt in contours:
        if unrot:
            x, y, w, h = cv.boundingRect(cnt)
            if filtre(w, h, dim):
                cv.rectangle(cimg, (x, y), (x + w, y + h), (0, 255 ,0), 3)
                good += 1
                rect.append([x, y, w, h])
        else:
            r = cv.minAreaRect(cnt)
            (cx, cy), (w, h), alpha = r
            if filtre(w, h, dim):
                box = cv.boxPoints(r)
                box = np.int0(box)
                cv.drawContours(cimg, [box], 0, (255, 0, 0), 3)
                good += 1
                rect.append([cx, cy, w, h, alpha])        
        
        total += 1

    if show:
        plt.imshow(cimg, cmap='Greys', aspect='equal')
        plt.axis('off')
    
    return good, total, np.array(rect)

In [4]:
def drawEllipse(image, show=True):
    dim = image.shape
    cimg = cv.cvtColor(image, cv.COLOR_GRAY2RGB)
    contours, h = cv.findContours(image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    elli = []
    
    total = 0
    good = 0
    
    for cnt in contours:
        
        if cnt.shape[0] > 4:
            el = cv.fitEllipse(cnt)
            (cx, cy), (a, b), alpha = el
            if filtre(a, b, dim):
                cv.ellipse(cimg, el, (255, 0, 0), 5)
                good += 1
                if b > a:
                    a, b = b, a
                    alpha = (alpha + 90)%180
                elli.append([cx, cy, a, b, alpha])    

            total += 1

    if show:
        plt.imshow(cimg, cmap='Greys', aspect='equal')
        plt.axis('off')
    
    return good, total, np.array(elli)

# Visualisation

In [5]:
plt.figure()

part_path = 'vert.bmp'
path = 'img/Reelles/' + part_path
img = cv.imread(path, 0)

def Rcallback(v: int, unrot: bool):
    ret, thresh = cv.threshold(img, v, 255, 0)
    if thresh.mean() < 127:
        thresh = 255 - thresh
    drawRectangle(thresh, True, unrot)
    
interact(Rcallback,
         v=widgets.IntSlider(
             min=0,
             max=255,
             step=1,
             value=100),
         unrot=widgets.Checkbox(
             value=False,
             description='Bloquer la rotation',
))

interactive(children=(IntSlider(value=100, description='v', max=255), Checkbox(value=False, description='Bloqu…

<function __main__.Rcallback(v: int, unrot: bool)>

In [6]:
plt.figure()

def Ecallback(v: int):
    ret, thresh = cv.threshold(img, v, 255, 0)
    if thresh.mean() < 127:
        thresh = 255 - thresh
    drawEllipse(thresh, True)
    
interact(Ecallback,
         v=widgets.IntSlider(
             min=0,
             max=255,
             step=1,
             value=100))

interactive(children=(IntSlider(value=100, description='v', max=255), Output()), _dom_classes=('widget-interac…

<function __main__.Ecallback(v: int)>

# Choix du seuil (picking the data)

In [7]:
n = 61
x = np.arange(80, 141)

In [8]:
def Radd(unrot):
    
    goods = np.zeros(n)
    totals = np.zeros(n)
    ratio = np.zeros(n)
    smallW = np.zeros(n)
    smallH = np.zeros(n)
    bigW = np.zeros(n)
    bigH = np.zeros(n)
    concentric = np.zeros(n)
    smallAngle = np.zeros(n)
    bigAngle = np.zeros(n)

    for i, v in enumerate(x):
        
        ret, thresh = cv.threshold(img, v, 255, 0)
        if thresh.mean() < 127:
            thresh = 255 - thresh
        a, b, rect = drawRectangle(thresh, False, unrot)
        if b == 0:
            ratio[i] = 0
        else:
            ratio[i] = a/b
        goods[i] = a
        totals[i] = b

        if a == 2:
            
            w = rect[:, 2]
            h = rect[:, 3]
            I = np.argmax(w*h) # on regarde le rectangle d'aire maximale
            smallW[i] = w[1 - I]
            bigW[i] = w[I]
            smallH[i] = h[1 - I]
            bigH[i] = h[I]
            
            if unrot:
                c = rect[:, :2] + .5*rect[:, 2:]
            else:
                c = rect[:, :2]
                angle = rect[:, 4]
                
                smallAngle[i] = angle[1 - I]
                bigAngle[i] = angle[I]
            concentric[i] = np.linalg.norm(c[1, :] - c[0, :])
            
    return goods, totals, ratio, smallW, smallH, bigW, bigH, concentric, smallAngle, bigAngle

In [9]:
def Eadd():
    
    goods = np.zeros(n)
    totals = np.zeros(n)
    ratio = np.zeros(n)
    smallA = np.zeros(n)
    smallB = np.zeros(n)
    bigA = np.zeros(n)
    bigB = np.zeros(n)
    concentric = np.zeros(n)
    smallAngle = np.zeros(n)
    bigAngle = np.zeros(n)

    for i, v in enumerate(x):
        
        ret, thresh = cv.threshold(img, v, 255, 0)
        if thresh.mean() < 127:
            thresh = 255 - thresh
        g, t, e = drawEllipse(thresh, False)
        if t == 0:
            ratio[i] = 0
        else:
            ratio[i] = g/t
        goods[i] = g
        totals[i] = t

        if g == 2:
            
            a = e[:, 2]
            b = e[:, 3]
            I = np.argmax(a)
            
            smallA[i] = a[1 - I]
            bigA[i] = a[I]
            smallB[i] = b[1 - I]
            bigB[i] = b[I]
            
            c = e[:, :2]
            smallAngle[i] = e[1 - I, 4]
            bigAngle[i] = e[I, 4]
            concentric[i] = np.linalg.norm(c[1, :] - c[0, :])
            
    return goods, totals, ratio, smallA, smallB, bigA, bigB, concentric, smallAngle, bigAngle

# Comparaison des méthodes

In [10]:
goods1, totals1, ratio1, smallW1, smallH1, bigW1, bigH1, concentric1, smallAngle1, bigAngle1 = Radd(True)
goods2, totals2, ratio2, smallW2, smallH2, bigW2, bigH2, concentric2, smallAngle2, bigAngle2 = Radd(False)
goods3, totals3, ratio3, smallA3, smallB3, bigA3, bigB3, concentric3, smallAngle3, bigAngle3 = Eadd()

In [11]:
fig, ax = plt.subplots(3, 3, figsize=(14, 7))

# Rectangles droits

ax[0, 0].plot(x, goods1)
ax[0, 0].set_title('Relevant rectangles (without rotation)')

ax[1, 0].plot(x, totals1)
ax[1, 0].set_title('Total number of rectangles detected (without rotation)')

ax[2, 0].plot(x, ratio1)
ax[2, 0].set_title('Ratio (without rotation)')

# Rectangles avec rotation

ax[0, 1].plot(x, goods2)
ax[0, 1].set_title('Relevant rectangles (with rotation)')

ax[1, 1].plot(x, totals2)
ax[1, 1].set_title('Total number of rectangles detected (with rotation)')

ax[2, 1].plot(x, ratio2)
ax[2, 1].set_title('Ratio (with rotation)')

# Ellipses

ax[0, 2].plot(x, goods2)
ax[0, 2].set_title('Relevant ellipses')

ax[1, 2].plot(x, totals2)
ax[1, 2].set_title('Total number of ellipses detected')

ax[2, 2].plot(x, ratio2)
ax[2, 2].set_title('Ratio (ellipses)')

for ax in ax.flat:
    ax.label_outer()
    
fig.tight_layout();

In [12]:
fig, ax = plt.subplots(3, 3, figsize=(14, 7))

mask1 = goods1 == 2
mask2 = goods2 == 2
mask3 = goods3 == 2
x1 = x[mask1]
x2 = x[mask2]
x3 = x[mask3]

# Rectangles droits

ax[0, 0].scatter(x1, smallW1[mask1], label='Width', s=5, marker='^')
ax[0, 0].scatter(x1, smallH1[mask1], label='Heigth', s=5, marker='v')
ax[0, 0].set_title('Small rectangle (without rotation)')
ax[0, 0].legend()

ax[1, 0].scatter(x1, bigW1[mask1], label='Width', s=5, marker='^')
ax[1, 0].scatter(x1, bigH1[mask1], label='Heigth', s=5, marker='v')
ax[1, 0].set_title('Big rectangle (without rotation)')
ax[1, 0].legend()

ax[2, 0].scatter(x1, concentric1[mask1], s=7, marker='x')
ax[2, 0].set_title('Concentricity (without rotation)')

# Rectangles avec rotation

ax[0, 1].scatter(x2, smallW2[mask2], label='Width', s=5, marker='^')
ax[0, 1].scatter(x2, smallH2[mask2], label='Heigth', s=5, marker='v')
ax[0, 1].set_title('Small rectangle (with rotation)')
ax[0, 1].legend()

ax[1, 1].scatter(x2, bigW2[mask2], label='Width', s=5, marker='^')
ax[1, 1].scatter(x2, bigH2[mask2], label='Heigth', s=5, marker='v')
ax[1, 1].set_title('Big rectangle (with rotation)')
ax[1, 1].legend()

ax[2, 1].scatter(x2, concentric2[mask2], s=7, marker='x')
ax[2, 1].set_title('Concentricity (with rotation)')

# Ellipses

ax[0, 2].scatter(x3, smallA3[mask3], label='Grand axe', s=5, marker='^')
ax[0, 2].scatter(x3, smallB3[mask3], label='Petit axe', s=5, marker='v')
ax[0, 2].set_title('Petite ellipse')
ax[0, 2].legend()

ax[1, 2].scatter(x3, bigA3[mask3], label='Grand axe', s=5, marker='^')
ax[1, 2].scatter(x3, bigB3[mask3], label='Petit axe', s=5, marker='v')
ax[1, 2].set_title('Grande ellipse')
ax[1, 2].legend()

ax[2, 2].scatter(x3, concentric3[mask3], s=7, marker='x')
ax[2, 2].set_title('Concentricity (ellipse)')

for ax in ax.flat:
    ax.label_outer()
    
fig.tight_layout();

In [13]:
# Angles de rotations

fig, ax = plt.subplots(1, 2, figsize=(12, 3))

ax[0].scatter(x2, smallAngle2[mask2], label='Petit angle', s=5, marker='^')
ax[0].scatter(x2, bigAngle2[mask2], label='Grand angle', s=5, marker='v')
ax[0].set_title('Angles (rectangles avec rotation)')
ax[0].legend()

ax[1].scatter(x3, smallAngle3[mask3], label='Petit angle', s=5, marker='^')
ax[1].scatter(x3, bigAngle3[mask3], label='Grand angle', s=5, marker='v')
ax[1].set_title('Angles (ellipse)')
ax[1].legend()

for ax in ax.flat:
    ax.label_outer()
    
fig.tight_layout();

# Reste à faire/tests unitaires

- Preprocessing (Canny, recadrage) ?
- Exploiter les mesures mises en évidence

_____

- Rectangles : centre, largeur, hauteur
- MinAreaRect : (centre), (largeur, hauteur), angle
- Ellipse : (centre), (grand axe, petit axe), angle